# Exercise 1 - Building a Decision Tree

First, we need to load in our dataset. We will be using the `spotifyclassification` (available [here](https://www.kaggle.com/datasets/geomack/spotifyclassification)) for this exercise.

Use the "Add Input" button in the panel on the right to make these datasets available to your notebook. If you have successfully added these datasets, then running the cell below should print the path you can use to load them in.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The cell below imports some packages that we will use for building and evaluating our models.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

## Reading in the data

Okay, we're ready to get started. Let's load in our first dataset and take a look at it!

In [ ]:
spotify_df = pd.read_csv("/kaggle/input/spotifyclassification/data.csv", index_col=0)
spotify_df.head()

The dataset contains a number of features of songs from 2017 available on Spotify, and a binary variable `target` that represents whether the user liked the song (encoded as 1) or not (encoded as 0).

These are *one* user's listening preferences. We will investigate whether the features Spotify provides are helpful in predicting whether or not a user likes a particular song.

---
### Questions: 
1. Look up the documentation of the features [here](https://developer.spotify.com/documentation/web-api/reference/get-audio-features). Which ones do you think are likely to help with predicting our **target** column? Which ones are best discarded?

2. Is this is supervised, unsupervised or self-supervised machine learning problem? Do we want to perform inference or prediction?

---

## Data Splitting

As always, our first task should be to split our data into a training and test set. We will not access the test data until the very end, when we are ready to evaluate our final trained model.

In [ ]:
# Split the data

train_df, test_df = train_test_split(spotify_df, test_size=0.2, random_state=123)

We can check how many data points our training and test sets contain

In [ ]:
n_train_samples = train_df.shape[0]
n_test_samples = test_df.shape[0]

print(f"Training Examples: {n_train_samples}")
print(f"Test Examples: {n_test_samples}")

Pandas also allows us to conveniently calculate some summary statistics for our dataset. For example, we can check what the mean and standard deviation for our numerical columns is.

In [ ]:
# Summary statistics

spotify_summary = train_df.describe()
spotify_summary

## Exploratory Data Analysis

We will first try to understand our dataset by examining some features directly. The code below produces histograms for the `loudness` feature using pandas plotting. The histograms show the distribution of the feature values in the training set, separated for positive (target=1, i.e., user liked the song) and negative (target=0, i.e., user disliked the song) examples.

There are two different histograms, one for target = 0 and one for target = 1, and they are overlaid on top of each other.

In [ ]:
# Plotting the loudness feature

feat = "loudness"
train_df.groupby("target")[feat].plot.hist(bins=50, alpha=0.5, legend=True, density = True, title = "Histogram of " + feat);
plt.xlabel(feat);


The histogram shows that extremely quiet songs tend to be disliked (more blue bars than orange on the left) and very loud songs also tend to be disliked (more blue than orange on the far right).

---
## Question:

Suppose we were to build a decision tree using only the `loudness` feature. What would a good decision tree look like? What threshold values is our model likely to learn? How deep should we make our tree?

---

## More plotting

The code below makes similar plots for some other features-- specifically `acousticness`, `danceability`, `tempo`, `energy` and `valence`. Run the cell and examine the histograms generated. Feel free to include more features in the list to be plotted.

In [ ]:
features = [
    "acousticness",
    "danceability",
    "tempo",
    "energy",
    "valence",
]
for feat in features:
    ax = train_df.groupby("target")[feat].plot.hist(bins=50, alpha=0.5, legend=True, density = True, title="Histogram of " + feat)
    plt.xlabel(feat)
    plt.show()


---
## Question:

Some features (like `tempo`) seem to have a similar distribution between both target classes (0 and 1). Should we drop these features from our data, based onthe histograms we plotted?

---

We are going to drop `song_title` and `artist` since they are text features. (Do you expect either feature to be useful?)

We are also separating the features from the target.

In [ ]:
X_train = train_df.drop(columns=["target", "song_title", "artist"])
y_train = train_df["target"]

X_test = test_df.drop(columns=["target", "song_title", "artist"])
y_test = test_df["target"]

## Dummy Classifier

First, we train a dummy classifier. This gives us a baseline prediction against which to compare our model performance.

The dummy classifier predicts the same value of the target for all examples in the training set. When we call `fit()`, our dummy classifier learns the most common value of the target. The function `score()` then predicts this value for all examples, and returns the proportion which were correctly predicted. So the value returned here is the proportion of examples that belong to the most common target class.

In [ ]:
# Dummy Classifier

dummy = DummyClassifier(random_state=123)
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

Looks like a balanced dataset! We have a close to 50-50 split between the target classes. Any ML model that actually learned useful information should perform with an accuracy higher than 50%.

We are now ready to build our decision tree.

In [ ]:
# Creating a decision tree

spotify_tree = DecisionTreeClassifier(random_state=123)

Let's train and score our tree.

In [ ]:
spotify_tree.fit(X_train, y_train)
spotify_tree.score(X_train, y_train)

That score is *very* high. Our model is classifying training examples with near-perfect accuracy! 

---
#### Question:

Do we trust our training scores to give a good prediction for model performance on unseen data? Have we built a good model?

---

## Examining our model

Let's see how deep the fitted decision tree looks.

In [ ]:
spotify_tree.get_depth()

---
### Question:

You should see a decision tree of depth 17. How many distinct training examples could a depth 17 decision tree memorize? How many examples exist in our dataset?

---

It would be good to know what features our tree thought were important! Let's visualize the first few nodes.

In [ ]:
# Training with just depth 3

three_model = DecisionTreeClassifier(max_depth=3, random_state=123)
three_model.fit(X_train, y_train)

In [ ]:
# Plotting

plot_tree(three_model, filled=True, 
               feature_names = X_train.columns.to_list(),
               impurity = False,
               fontsize = 7,
               label = None);

___

### Question:

What features do you see the tree using? Do they match the features you expected to see based on the histograms in your EDA?

___

Our model achieved a near-perfect training score. However, since we trained on all data in our training set, we have not yet been able to measure model performance on new, unseen data.

Let's check how well our trained model does on the test set.

In [ ]:
spotify_tree.score(X_test, y_test)

Hmmm. That wasn't nearly as good. We went from perfect predictions on training data to only 68% accuracy on test data. It is likely that we have overfit.

## Finding a better model

We fit a very deep decision tree that likely just memorized our dataset. A shallower tree might limit some of the overfitting, so let's try forcing a maximum depth on our tree.

In [ ]:
new_model = DecisionTreeClassifier(max_depth=5, random_state=123)

We train and score this model as usual on the training set.

In [ ]:
new_model.fit(X_train, y_train)
new_model.score(X_train, y_train)

We get a lower training score than before. But what about test score?

In [ ]:
new_model.score(X_test, y_test)

There's still some overfitting, but we're getting a better test score than what we got before (with a deeper, more complex decision tree!) Also, there is less of a gap between our training and test scores, so our model is generalizating reasonably to new data, and making sensible predictions.

## Hold on... did we violate the Golden Rule?

We used our test set twice, which means we violated the Golden Rule of Machine Learning. In practice, it is *not* a good idea to use our test set multiple times with different models. The more often we use it, the less certain we can be of our model's predictions out in the real world. We might start overfitting to test data, even though we don't use it to directly update model parameters.

This is a real danger, and can lead us to picking bad models.

___
### Question:

If re-using the test set is bad, but we cannot evaluate model performance until we measure it on unseen data, how are we to select good values for hyperparameters or choose between models?

How do we find the best `max_depth` for modelling our Spotify data?

___

Hyperparameter tuning is generally carried out using **cross-validation**. Essentially, this involves setting aside part of your training data as a 'validation set' to be used *only* for testing (and not for fitting each individual model). Then, validation accuracy can be used as a proxy for test accuracy.

Sklearn has many helper functions to make cross-validation easy (see [here](https://scikit-learn.org/1.5/modules/cross_validation.html#cross-validation-and-model-selection)).

## Feature Importance

Sklearn also has a built-in function to tell you which features the decision tree *thinks* are most important for making predictions (based on its learning).

In [ ]:
impt = new_model.feature_importances_
for i, f in enumerate(X_train.columns.to_list()):
    print(f"{f}: ", round(impt[i], 4))

## Food for thought... (Bonus Question)

The code below takes the "index" column of this dataset and allows us to use it as a feature. What might be the result of using the index as a feature for this dataset?

In [ ]:
X_train_2 = X_train.reset_index()

In [ ]:
index_model = DecisionTreeClassifier(random_state=123)
index_model.fit(X_train_2, y_train)

In [ ]:
plot_tree(index_model, impurity=False, filled=True);

That's a very simple tree... what happened?